In [42]:
import gmsh
import os
import sys
import math


# Stage 0

In [43]:
# A cube

gmsh.initialize()
gmsh.model.add("t1")


lc = 0.02 # lc is the size


gmsh.model.geo.addPoint(0, 0, 0, lc)
gmsh.model.geo.addPoint(1, 0, 0, lc)
gmsh.model.geo.addPoint(1, 1, 0, lc)
gmsh.model.geo.addPoint(0, 1, 0, lc)

gmsh.model.geo.addLine(1,2)
gmsh.model.geo.addLine(2,3)
gmsh.model.geo.addLine(3,4)
gmsh.model.geo.addLine(4,1)

gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)

gmsh.model.geo.addPlaneSurface([1], 1)

extruded = gmsh.model.geo.extrude([(2,1)], 0, 0, 1)


gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(3)      
gmsh.write("cube.msh")         

gmsh.finalize()



Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 60%] Meshing curve 9 (Line)
Info    : [ 70%] Meshing curve 11 (Line)
Info    : [ 80%] Meshing curve 12 (Line)
Info    : [ 90%] Meshing curve 16 (Line)
Info    : [100%] Meshing curve 20 (Line)
Info    : Done meshing 1D (Wall 0.00192469s, CPU 0.002144s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 13 (Surface, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 17 (Surface, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 21 (Surface, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 25 (Surface, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 26 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wa

In [44]:
# A circle



gmsh.initialize()

ls = 0.02

gmsh.model.geo.addPoint(0,0,0, ls, 1)

gmsh.model.geo.addPoint(1,0,0, ls, 2)
gmsh.model.geo.addPoint(0,1,0, ls, 3)
gmsh.model.geo.addPoint(-1,0,0, ls, 4)
gmsh.model.geo.addPoint(0,-1,0, ls, 5)

gmsh.model.geo.addCircleArc(2, 1, 3)
gmsh.model.geo.addCircleArc(3, 1, 4)
gmsh.model.geo.addCircleArc(4, 1, 5)
gmsh.model.geo.addCircleArc(5, 1, 2)


gmsh.model.geo.addCurveLoop([1,2,3,4], 1)
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.geo.synchronize()

gmsh.model.mesh.generate(2)     
gmsh.write("circle.msh")          
gmsh.finalize()



Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 30%] Meshing curve 2 (Circle)
Info    : [ 60%] Meshing curve 3 (Circle)
Info    : [ 80%] Meshing curve 4 (Circle)
Info    : Done meshing 1D (Wall 0.000767036s, CPU 0.001203s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.28926s, CPU 0.29073s)
Info    : 9399 nodes 18799 elements
Info    : Writing 'circle.msh'...
Info    : Done writing 'circle.msh'


In [45]:
# cylinder
gmsh.initialize()

ls = 0.02

gmsh.model.geo.addPoint(0,0,0, ls, 1)

gmsh.model.geo.addPoint(1,0,0, ls, 2)
gmsh.model.geo.addPoint(0,1,0, ls, 3)
gmsh.model.geo.addPoint(-1,0,0, ls, 4)
gmsh.model.geo.addPoint(0,-1,0, ls, 5)

gmsh.model.geo.addCircleArc(2, 1, 3)
gmsh.model.geo.addCircleArc(3, 1, 4)
gmsh.model.geo.addCircleArc(4, 1, 5)
gmsh.model.geo.addCircleArc(5, 1, 2)


gmsh.model.geo.addCurveLoop([1,2,3,4], 1)
gmsh.model.geo.addPlaneSurface([1], 1)

extruded = gmsh.model.geo.extrude([(2,1)], 0, 0, 1)

gmsh.model.geo.synchronize()


gmsh.model.mesh.generate(3)    
gmsh.write("cylinder.msh")          

gmsh.finalize()



Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 20%] Meshing curve 3 (Circle)
Info    : [ 30%] Meshing curve 4 (Circle)
Info    : [ 40%] Meshing curve 6 (Circle)
Info    : [ 50%] Meshing curve 7 (Circle)
Info    : [ 60%] Meshing curve 8 (Circle)
Info    : [ 60%] Meshing curve 9 (Circle)
Info    : [ 70%] Meshing curve 11 (Line)
Info    : [ 80%] Meshing curve 12 (Line)
Info    : [ 90%] Meshing curve 16 (Line)
Info    : [100%] Meshing curve 20 (Line)
Info    : Done meshing 1D (Wall 0.00155958s, CPU 0.002346s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 13 (Surface, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 17 (Surface, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 21 (Surface, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 25 (Surface, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 26 (Plane, Frontal-Delaunay)
Info    : Don

# Stage 1 (thor)

In [46]:
gmsh.initialize()


gmsh.merge("thor_with_empty_soul.stl")

field = gmsh.model.mesh.field.add("MathEval")
gmsh.model.mesh.field.setString(field, "F", "3")
gmsh.model.mesh.field.setAsBackgroundMesh(field)


gmsh.model.mesh.generate(3)
gmsh.write('thor.msh')


gmsh.finalize()


Info    : Reading 'thor_with_empty_soul.stl'...
Info    : Mesh is in binary format
Info    : 4688 facets in solid 0                                                                                
Info    : Done reading 'thor_with_empty_soul.stl'
Info    : Meshing 1D...
Info    : Done meshing 1D (Wall 8.75701e-06s, CPU 1e-05s)
Info    : Meshing 2D...
Info    : Done meshing 2D (Wall 2.2593e-05s, CPU 2.2e-05s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 4.348e-06s, CPU 4e-06s)
Info    : 2344 nodes 4688 elements
Info    : Writing 'thor.msh'...
Info    : Done writing 'thor.msh'


# Stage 2 (crabbie)

In [47]:
gmsh.initialize()


gmsh.merge("headcrab.stl")

field = gmsh.model.mesh.field.add("MathEval")
gmsh.model.mesh.field.setString(field, "F", "0.2")
gmsh.model.mesh.field.setAsBackgroundMesh(field)


gmsh.model.mesh.generate(3)
gmsh.write('crabbie.msh')


gmsh.finalize()


Info    : Reading 'headcrab.stl'...
Info    : 1928 facets in solid 0 Default
Info    : Done reading 'headcrab.stl'
Info    : Meshing 1D...
Info    : Done meshing 1D (Wall 8.65599e-06s, CPU 1e-05s)
Info    : Meshing 2D...
Info    : Done meshing 2D (Wall 2.5609e-05s, CPU 2.5e-05s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 4.107e-06s, CPU 4e-06s)
Info    : 1000 nodes 1928 elements
Info    : Writing 'crabbie.msh'...
Info    : Done writing 'crabbie.msh'
